In [12]:
import xmltodict

# Import spaCy and load a language model
import spacy
nlp = spacy.load("en_core_web_sm")

training = 'ABSA16_Restaurants_Train_SB1_v2.xml'
testing = 'EN_REST_SB1_TEST.xml.gold'

with open(testing) as fd:
    xml_source = xmltodict.parse(fd.read())

In [13]:
xml_source

OrderedDict([('Reviews',
              OrderedDict([('Review',
                            [OrderedDict([('@rid',
                                           'en_BlueRibbonSushi_478218171'),
                                          ('sentences',
                                           OrderedDict([('sentence',
                                                         [OrderedDict([('@id',
                                                                        'en_BlueRibbonSushi_478218171:0'),
                                                                       ('text',
                                                                        'Yum!'),
                                                                       ('Opinions',
                                                                        OrderedDict([('Opinion',
                                                                                      OrderedDict([('@target',
                                              

In [14]:
xml_source.values()

odict_values([OrderedDict([('Review', [OrderedDict([('@rid', 'en_BlueRibbonSushi_478218171'), ('sentences', OrderedDict([('sentence', [OrderedDict([('@id', 'en_BlueRibbonSushi_478218171:0'), ('text', 'Yum!'), ('Opinions', OrderedDict([('Opinion', OrderedDict([('@target', 'NULL'), ('@category', 'FOOD#QUALITY'), ('@polarity', 'positive'), ('@from', '0'), ('@to', '0')]))]))]), OrderedDict([('@id', 'en_BlueRibbonSushi_478218171:1'), ('text', 'Serves really good sushi.'), ('Opinions', OrderedDict([('Opinion', OrderedDict([('@target', 'sushi'), ('@category', 'FOOD#QUALITY'), ('@polarity', 'positive'), ('@from', '19'), ('@to', '24')]))]))]), OrderedDict([('@id', 'en_BlueRibbonSushi_478218171:2'), ('text', 'Not the biggest portions but adequate.'), ('Opinions', OrderedDict([('Opinion', OrderedDict([('@target', 'portions'), ('@category', 'FOOD#STYLE_OPTIONS'), ('@polarity', 'neutral'), ('@from', '16'), ('@to', '24')]))]))]), OrderedDict([('@id', 'en_BlueRibbonSushi_478218171:3'), ('text', 'Gree

In [15]:
for reviews in xml_source['Reviews']['Review']:
    for sentences in reviews['sentences']['sentence']:
        try:
            # print(sentences['@id'], sentences['text'])
            sentences_text = sentences['text'].replace('&apos;', '  ')
            # print(sentences_text)
            for opini in sentences['Opinions'].values():
                # print(sentences_text)

                if isinstance(opini, dict):
                    opini = [opini]
                
                # print(opini)

                for each in opini:
                    start_position = int(each['@from'])
                    end_position = int(each['@to'])
                    
                    # print(sentences_text[:start_position]+"$T$"+sentences_text[end_position:])
                    
                    doc = nlp(sentences_text)
                    
                    # combine aspect if they were more than one token
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc.char_span(start_position, end_position))

                    # select span of aspect
                    aspect_token = doc.char_span(start_position, end_position)
                    
                    for token in doc:
                        if token.idx != start_position:
                            print(token.text+" ", end="")
                        else:
                            print("$T$ ", end="")
                    print()
                    print(each['@target'])
                    if each['@polarity'] == 'negative':
                        print("-1")
                    elif each['@polarity'] == 'positive':
                        print("1")
                    else:
                        print("0")

        except:
            continue

Serves really good $T$ . 
sushi
1
Not the biggest $T$ but adequate . 
portions
0
$T$ is a must ! 
Green Tea creme brulee
1
It has great $T$ and even better service . 
sushi
1
It has great sushi and even better $T$ . 
service
1
The entire $T$ was extremely accomodating and tended to my every need . 
staff
1
I 've been to this $T$ over a dozen times with no complaints to date . 
restaurant
1
The $T$ is belligerent to guests that have a complaint . 
owner
-1
Good $T$ ! 
food
1
This is a great place to get a delicious $T$ . 
meal
1
The $T$ is pretty friendly . 
staff
1
The $T$ are great ! 
onion rings
1
The $T$ tasted like sticky sweet donuts and the honey walnut prawns , the few they actually give you ..... were not good . 
lemon chicken
-1
The lemon chicken tasted like sticky sweet donuts and the $T$ , the few they actually give you ..... were not good . 
honey walnut prawns
-1
The lemon chicken tasted like sticky sweet donuts and the $T$ , the few they actually give you ..... were not g

In [16]:
# Import spaCy and load a language model
import spacy
nlp = spacy.load("en_core_web_sm")

# Create a spaCy document object from the input sentence
sentence = "He likes food"
doc = nlp(sentence)

# Iterate over the tokens and print their text, tag, and dep
for token in doc:
    print(token.text, token.tag_, token.dep_)

# Define a function that returns the tree-based distance between two tokens
def tree_distance(token1, token2):
    # Initialize a set to store the ancestors of token1
    ancestors1 = set()
    # Traverse the dependency tree from token1 to the root
    while token1 != token1.head:
        # Add the current token to the set
        ancestors1.add(token1)
        # Move to the parent token
        token1 = token1.head
    # Add the root token to the set
    ancestors1.add(token1)

    # Initialize a variable to store the distance
    distance = 0
    # Traverse the dependency tree from token2 to the root
    while token2 != token2.head:
        # If token2 is in the set of ancestors of token1, return the distance
        if token2 in ancestors1:
            return distance
        # Otherwise, increment the distance and move to the parent token
        else:
            distance += 1
            token2 = token2.head
    # If token2 is the root, return the distance
    return float(distance)

# Call the function with any two tokens and print the result
token1 = doc[0] # He
token2 = doc[2] # food
print("The tree-based distance between", token1.text, "and", token2.text, "is", tree_distance(token1, token2))


He PRP nsubj
likes VBZ ROOT
food NN dobj
The tree-based distance between He and food is 1.0


In [17]:
def compute_tree_based_distance(sentence, aspect, context_word):
    """Computes the tree-based distance between two words in a sentence.

    Args:
        sentence: A spaCy sentence object.
        aspect: The aspect word.
        context_word: The context word.

    Returns:
        The tree-based distance between the two words.
    """

    # Find the dependency tree for the sentence.
    dependency_tree = sentence.to_dependency_parse()

    # Find the shortest path between the aspect word and the context word in the dependency tree.
    path = dependency_tree.shortest_path(aspect, context_word)

    # Return the length of the shortest path.
    return len(path)

In [18]:
for reviews in xml_source['Reviews']['Review']:
    for sentences in reviews['sentences']['sentence']:
        try:
            # print(sentences['@id'], sentences['text'])
            sentences_text = sentences['text']
            # print(sentences_text)
            for opini in sentences['Opinions'].values():
                # print(sentences_text)

                if isinstance(opini, dict):
                    opini = [opini]
                
                # print(opini)

                for each in opini:
                    start_position = int(each['@from'])
                    end_position = int(each['@to'])
                    
                    # print(sentences_text[:start_position]+"$T$"+sentences_text[end_position:])
                    
                    doc = nlp(sentences_text)
                    
                    # combine aspect if they were more than one token
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc.char_span(start_position, end_position))

                    # select span of aspect
                    aspect_token = doc.char_span(start_position, end_position)
                    
                    for token in doc:
                        if token.idx != start_position:
                            print(token.text+" ", end="")
                        else:
                            print("$T$ ", end="")
                    print()

                    # print(doc.text[:start_position]+"$T$"+doc.text[end_position:])

                    # for each token in the sentence
                    for token in doc:
                        # for every word in the aspect token
                        distance_score = []
                        for aspect in aspect_token:
                            # find tree distance between the token with the word
                            score = tree_distance(token, aspect)
                            # score = compute_tree_based_distance(doc, token, aspect)
                            # save the distance in a list
                            distance_score.append(score)
                        # print only the smallest distance    
                        print(min(distance_score), end=" ")
                    print()

                    


        except:
            continue
        

Serves really good $T$ . 
1.0 0 0 0 1.0 
Not the biggest $T$ but adequate . 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 
$T$ is a must ! 
0 1.0 1.0 1.0 1.0 
It has great $T$ and even better service . 
1.0 1.0 0 0 0 0 0 0 1.0 
It has great sushi and even better $T$ . 
2.0 2.0 1 1 1 0 0 0 2.0 
The entire $T$ was extremely accomodating and tended to my every need . 
0 0 0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 
I 've been to this $T$ over a dozen times with no complaints to date . 
2.0 2.0 2.0 1 0 0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 
The $T$ is belligerent to guests that have a complaint . 
0 0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 
Good $T$ ! 
0.0 0.0 0.0 
This is a great place to get a delicious $T$ . 
3.0 3.0 2 2 2 1 1 0 0 0 3.0 
The $T$ is pretty friendly . 
0 0 1.0 1.0 1.0 1.0 
The $T$ are great ! 
0 0 1.0 1.0 1.0 
The $T$ tasted like sticky sweet donuts and the honey walnut prawns , the few they actually give you ..... were not good . 
0 0 1 1 1 1 1 1 1 1 1 1 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2

In [22]:
fname = './datasets/semeval16/restaurant_2016_training_coba_coba.xml.seg'
# fname = './datasets/semeval14/Restaurants_Train.xml.seg'
fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
lines = fin.readlines()
fin.close()
fin = open(fname+'.dist', 'r', encoding='utf-8', newline='\n', errors='ignore')
dist_lines = fin.readlines()
fin.close()

print(len(lines))

# print(dist_lines[5114])
# print(dist_lines[5115])


# cnt = 0
# for i in range(0, len(lines), 3):
#     print(cnt*2+1)
#     # print(dependency_dist[i])

#     print([float(d) for d in dist_lines[cnt*2+1].split()])

#     dependency_dist = [float(d) for d in dist_lines[cnt*2+1].split()]
#     cnt += 1

cnt = 0
jumlah_sequence = []
for i in range(0, len(lines), 3):
    try:
        # Print the element at index cnt*2+1
  
        # print([float(d) for d in dist_lines[cnt*2+1].split()])
        # Assign the element to dependency_dist
        dependency_dist = [float(d) for d in dist_lines[cnt*2+1].split()]
        # print(len(dependency_dist))
        # Increment cnt by 1
        cnt += 1
        jumlah_sequence.append(len(dependency_dist))
    except IndexError:
        # Print an error message and break the loop
        print("Index out of range: {}".format(cnt*2+1))
        break
print(max(jumlah_sequence))
    


5445
74
